## 1.Download Dataset

In [1]:
#import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [2]:
#import the canada_coordinate.csv table as previously created
canada_df = pd.read_csv('canada_coordinate.csv')
canada_df.head(10) #display the first 10 rows from the table

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626
4,M6A,North York,Lawrence Heights,43.7223,-79.4504
5,M6A,North York,Lawrence Manor,43.7223,-79.4504
6,M7A,Queen's Park,Queen's Park,43.6641,-79.3889
7,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
8,M1B,Scarborough,Rouge,43.8113,-79.1930
9,M1B,Scarborough,Malvern,43.8113,-79.1930


## 2.Creating Toronto Dataframe

#### For this exercise, only neighborhoods in Toronto is explored. So, the original dataframe is sliced to create new dataframe of Toronto.

In [3]:
# explore the all the Borough list
canada_df['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [4]:
# creating a list of Toronto from the Borough
toronto = ['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [5]:
#creating Toronto dataframe from the original dataframe by using the Toronto list created
toronto_df = canada_df[canada_df.Borough.isin(toronto)].reset_index(drop = True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
1,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3783
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3783
4,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
5,M4E,East Toronto,The Beaches,43.6784,-79.2941
6,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754
7,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860
8,M6G,Downtown Toronto,Christie,43.6683,-79.4205
9,M5H,Downtown Toronto,Adelaide,43.6496,-79.3833


#### Checking the dataframe size

In [6]:
toronto_df.shape 

(74, 5)

#### Getting Geographical Coordinate for Toronto

In [7]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Geography coordinate of Toronto is {}, {}'.format(latitude, longitude))

Geography coordinate of Toronto is 43.653963, -79.387207


#### Visualizing Neighborhoods in Toronto

In [8]:
# create toronto map
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to the toronto map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lng],
                        radius = 5,
                        popup = label,
                        color = 'blue',
                        fill = True,
                        fill_color = '#3186cc',
                        fill_opacity = 0.7,
                        parse_html = False,
                       ).add_to(map_toronto)
map_toronto

## 3.Explore Neighborhoods in Toronto

#### Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'XQYTD01WFJPCVBX01GCGY5GJWJ5DH05PIPAZEHJFVESOVNR0'
CLIENT_SECRET = 'BI2NPYJKK0FOBFTDDFT0DPFW2YDKEOFEP10J4R1VE5SF4PL5'
VERSION = '20180605'

print('Client_ID: {}, Client_secret: {}, Version: {}'.format(CLIENT_ID, CLIENT_SECRET, VERSION))

Client_ID: XQYTD01WFJPCVBX01GCGY5GJWJ5DH05PIPAZEHJFVESOVNR0, Client_secret: BI2NPYJKK0FOBFTDDFT0DPFW2YDKEOFEP10J4R1VE5SF4PL5, Version: 20180605


#### create a function to get nearby venue

In [10]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list =[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #create API request for URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT
        )
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevent information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
    nearby_venues.columns = ['Neighborhood',
                                 'Neighborhood Latitude',
                                 'Neighborhood Longitude',
                                 'Venue',
                                 'Venue Latitude',
                                 'Venue Longitude',
                                 'Venue Category'
                                ]
        
    return (nearby_venues)

#### Create new dataframe *toronto_venue* from the above function

In [11]:
toronto_venues = getNearbyVenues(names = toronto_df['Neighborhood'], 
                                 latitudes = toronto_df['Latitude'], 
                                 longitudes = toronto_df['Longitude'])

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


#### Checking the dataframe size and the new dataframe created

In [12]:
print(toronto_venues.shape)
toronto_venues.head(10)

(3293, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,Harbourfront,43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
5,Harbourfront,43.6555,-79.3626,Cocina Economica,43.654959,-79.365657,Mexican Restaurant
6,Harbourfront,43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
7,Harbourfront,43.6555,-79.3626,Sumach Espresso,43.658135,-79.359515,Coffee Shop
8,Harbourfront,43.6555,-79.3626,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
9,Harbourfront,43.6555,-79.3626,Berkeley Church,43.655123,-79.365873,Event Space


#### Checking numbers of venues returned for each Neighborhood

In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,57,57,57,57,57,57
Berczy Park,93,93,93,93,93,93
Brockton,38,38,38,38,38,38
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
CN Tower,57,57,57,57,57,57
Cabbagetown,45,45,45,45,45,45
Central Bay Street,93,93,93,93,93,93
Chinatown,100,100,100,100,100,100


#### Check how many unique categories can be curated from all the returned venues

In [14]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 221 unique categories


## 4.Analyze Each Neighborhood

In [15]:
#using onehot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot.drop('Neighborhood', axis = 1, inplace = True)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] # adding neighborhood back to the dataframe

# move the neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(10)

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Checking the dataframe size

In [16]:
toronto_onehot.shape

(3293, 221)

#### Group the row by Neighborhood and calculating mean of the frequency occurance foreach category

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Adelaide,0.000000,0.0,0.030000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.030000,0.0,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.0,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.01,0.000000,0.000000,0.070000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.060000,0.0,0.000000,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.020000,0.010000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.01,0.0,0.030000,0.0,0.000000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.040000,0.000000,0

#### Checking the dataframe size

In [18]:
toronto_grouped.shape

(73, 221)

#### Print each Neighborhood with the 5 top most common venues

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("---"+hood+"---")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Adelaide---
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2        Hotel  0.04
3   Steakhouse  0.04
4    Gastropub  0.03


---Bathurst Quay---
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.07
2          Restaurant  0.05
3                Café  0.05
4                 Bar  0.05


---Berczy Park---
         venue  freq
0  Coffee Shop  0.11
1        Hotel  0.05
2         Café  0.05
3   Restaurant  0.04
4       Bakery  0.04


---Brockton---
                    venue  freq
0             Coffee Shop  0.08
1          Breakfast Spot  0.05
2                    Café  0.05
3  Thrift / Vintage Store  0.05
4       Accessories Store  0.03


---Business Reply Mail Processing Centre 969 Eastern---
               venue  freq
0        Coffee Shop  0.12
1        Yoga Studio  0.06
2         Restaurant  0.06
3        Gaming Cafe  0.06
4  Martial Arts Dojo  0.06


---CN Tower---
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant

#### Create funtion to sort the venues in ascending order

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create dataframe that show the 10 most common venues of each neighborhood

In [21]:
# set number of venues to create
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to the number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create new dataframe 
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Café,Coffee Shop,Hotel,Steakhouse,Thai Restaurant,Burger Joint,Restaurant,Japanese Restaurant,Gym,Bar
1,Bathurst Quay,Coffee Shop,Italian Restaurant,Bar,Restaurant,Café,Sandwich Place,Speakeasy,Gym / Fitness Center,Bakery,Pub
2,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Bakery,Seafood Restaurant,Beer Bar,Italian Restaurant,Cocktail Bar,Farmers Market
3,Brockton,Coffee Shop,Breakfast Spot,Café,Thrift / Vintage Store,Diner,Brewery,Caribbean Restaurant,Sandwich Place,Chiropractor,Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Yoga Studio,Breakfast Spot,Bookstore,Salad Place,Sushi Restaurant,Gaming Cafe,Road,Restaurant,Japanese Restaurant
5,CN Tower,Coffee Shop,Italian Restaurant,Bar,Restaurant,Café,Sandwich Place,Speakeasy,Gym / Fitness Center,Bakery,Pub
6,Cabbagetown,Pizza Place,Coffee Shop,Bakery,Park,Café,Market,Italian Restaurant,Restaurant,Caribbean Restaurant,Breakfast Spot
7,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Clothing Store,Italian Restaurant,Tea Room,Café,Breakfast Spot,Fast Food Restaurant,Gym / Fitness Center,Bubble Tea Shop
8,Chinatown,Café,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Dumpling Restaurant,Bakery,Coffee Shop,Burger Joint
9,Christie,Café,Grocery Store,Coffee Shop,Playground,Park,Athletics & Sports,Baby Store,Falafel Restaurant,Eastern European Restaurant,Electronics Store


## 5.Clustering The Neighborhood

#### Using K-Means to cluster Neighborhood into 5 clusters

In [22]:
#set number od clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis = 1)

# run the kmean clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# check the clutsering cloumn created for top 10 row
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

#### Create dataframe which include clustering label and top 10 most common venues for each neighborhood

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_df with the neighborhpood_venues_sorted to get the latitude and longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')

toronto_merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626,1,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Pub,Restaurant,Electronics Store,Event Space
1,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626,1,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Pub,Restaurant,Electronics Store,Event Space
2,M5B,Downtown Toronto,Ryerson,43.6572,-79.3783,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Pizza Place,Bookstore,Restaurant,Ramen Restaurant,Plaza
3,M5B,Downtown Toronto,Garden District,43.6572,-79.3783,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Pizza Place,Bookstore,Restaurant,Ramen Restaurant,Plaza
4,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,1,Coffee Shop,Café,Restaurant,Bakery,Pizza Place,Hotel,Italian Restaurant,Seafood Restaurant,Breakfast Spot,Park
5,M4E,East Toronto,The Beaches,43.6784,-79.2941,1,Pub,Trail,Gastropub,Health Food Store,Bakery,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio
6,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,1,Coffee Shop,Hotel,Café,Restaurant,Bakery,Seafood Restaurant,Beer Bar,Italian Restaurant,Cocktail Bar,Farmers Market
7,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860,1,Coffee Shop,Middle Eastern Restaurant,Clothing Store,Italian Restaurant,Tea Room,Café,Breakfast Spot,Fast Food Restaurant,Gym / Fitness Center,Bubble Tea Shop
8,M6G,Downtown Toronto,Christie,43.6683,-79.4205,1,Café,Grocery Store,Coffee Shop,Playground,Park,Athletics & Sports,Baby Store,Falafel Restaurant,Eastern European Restaurant,Electronics Store
9,M5H,Downtown Toronto,Adelaide,43.6496,-79.3833,1,Café,Coffee Shop,Hotel,Steakhouse,Thai Restaurant,Burger Joint,Restaurant,Japanese Restaurant,Gym,Bar


#### Visualizing the map of the resulting clusters

In [27]:
# create map
map_clusters = folium.Map(location = [latitude,longitude], zoom_start=11)

#setting the color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html= True)
    folium.CircleMarker ([lat, lon],
                         radius = 5,
                         popup = label,
                         color = rainbow[cluster - 1],
                         fill = True,
                         fill_color = rainbow[cluster - 1],
                         fill_opacity = 0.7
                        ).add_to(map_clusters)
    
map_clusters

## 6.Examining Each Cluster

#### Examining each cluster to determine the venues categories that distinguish each cluster.

### Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Central Toronto,0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
55,Central Toronto,0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
56,Central Toronto,0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
57,Central Toronto,0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
58,Central Toronto,0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


### Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Pub,Restaurant,Electronics Store,Event Space
1,Downtown Toronto,1,Coffee Shop,Gym / Fitness Center,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Pub,Restaurant,Electronics Store,Event Space
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Pizza Place,Bookstore,Restaurant,Ramen Restaurant,Plaza
3,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Pizza Place,Bookstore,Restaurant,Ramen Restaurant,Plaza
4,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Pizza Place,Hotel,Italian Restaurant,Seafood Restaurant,Breakfast Spot,Park
5,East Toronto,1,Pub,Trail,Gastropub,Health Food Store,Bakery,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio
6,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Bakery,Seafood Restaurant,Beer Bar,Italian Restaurant,Cocktail Bar,Farmers Market
7,Downtown Toronto,1,Coffee Shop,Middle Eastern Restaurant,Clothing Store,Italian Restaurant,Tea Room,Café,Breakfast Spot,Fast Food Restaurant,Gym / Fitness Center,Bubble Tea Shop
8,Downtown Toronto,1,Café,Grocery Store,Coffee Shop,Playground,Park,Athletics & Sports,Baby Store,Falafel Restaurant,Eastern European Restaurant,Electronics Store
9,Downtown Toronto,1,Café,Coffee Shop,Hotel,Steakhouse,Thai Restaurant,Burger Joint,Restaurant,Japanese Restaurant,Gym,Bar


### Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,West Toronto,2,Park,Yoga Studio,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
37,West Toronto,2,Park,Yoga Studio,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### Cluster 4

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Central Toronto,3,Home Service,Spa,Dumpling Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
34,Central Toronto,3,Home Service,Gaming Cafe,Trail,Electronics Store,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
35,Central Toronto,3,Home Service,Gaming Cafe,Trail,Electronics Store,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


### Cluster 5

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,4,Park,Harbor / Marina,Café,Music Venue,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
15,Downtown Toronto,4,Park,Harbor / Marina,Café,Music Venue,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
16,Downtown Toronto,4,Park,Harbor / Marina,Café,Music Venue,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
31,Central Toronto,4,Photography Studio,Park,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
38,Central Toronto,4,Playground,Gym Pool,Park,Garden,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
49,Central Toronto,4,Park,Grocery Store,Tennis Court,Thai Restaurant,Gym,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio
50,Central Toronto,4,Park,Grocery Store,Tennis Court,Thai Restaurant,Gym,Event Space,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio
66,Downtown Toronto,4,Playground,Park,Candy Store,Grocery Store,Bank,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant
